In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import preprocesing as pre
from configparser import ConfigParser
from helpers import plot_MinMaxAvg

In [ ]:
from helpers import show_classes_distribution

# Creating dataset

* Zrodlo danych: Sentinel
* Konwerscja JP2 do PNG: ręcznie / skrypt w bash
* Klasyfikacja terenu na podstawie Open Street Map: natural water, landuse forest, landuse farmland
* Ręczna poprawa klasyfikacji

Reading project variable from config.ini file

In [ ]:
config = ConfigParser()
config.read('config.ini')
input_dir = config['main']['input_dir']
class_file = config['main']['classification_data']
dx = int(config['main']['x_size'])
dy = int(config['main']['y_size'])
x = int(config['main']['x_start'])
y = int(config['main']['y_start'])
csv_data_file = config['main']['csv_data_file']

Read files from data folder

In [ ]:
data, columns_names = pre.images_to_numpy(input_dir, dx, dy, x, y)

In [ ]:
plot_MinMaxAvg(data)

the data has a large variance so we have to deal with outstanding data $ x = min(x,\overline{x}+3\sigma_{x}) $ and implement minimax scaller from sklearn. 
* Normalisation should be done on training and test separetly but the project is just for learning purpose

In [ ]:
 data = pre.remove_outstandings(data)

In [ ]:
plot_MinMaxAvg(data)

Read classes from directory

In [ ]:
classes, class_names = pre.get_classes(class_file, dx, dy, x, y)
other = (1 - classes.any(axis=1).astype(int)).reshape(-1,1)
class_names += ['other']
pre.add_classes_to_config(config, class_names)
columns_names += class_names 

In [ ]:
show_classes_distribution(np.concatenate((classes, other), axis=1),class_names)

Saving data to csv file

In [ ]:
data = np.concatenate((data, classes, other), axis=1)
data = pd.DataFrame(data, columns=columns_names)
data.to_csv(csv_data_file)